# Вариационный автокодировщик

В задании предлагается реализовать вариационный и обычный автокодировщики, обучить их на MNIST, сравнить между собой эти модели, сделать выводы по результатам сравнения и выводы про каждую модель по отдельности.

Необходимая теория приведена ниже. Для более глубокого погружения в тему также есть список литературы с комментариями.

В этом задании нельзя использовать функций плотностей распределений, KL-дивергенцию и репараметризации из стандартных библиотек.

Актуальная версия доступна по адресу https://github.com/nadiinchi/dl_labs/blob/master/ht_vae/experiments.ipynb

## Теория

### Постановка задачи
Дана выборка независимых одинаково распределенных величин из истинного распределения $x_i \sim p_d(x)$, $i = 1, \dots, N$.

Задача - построить вероятностную модель $p_\theta(x)$ истинного распределения $p_d(x)$.

Распределение $p_\theta(x)$ должно позволять как оценить плотность вероятности для данного объекта $x$, так и сэмплировать $x \sim p_\theta(x)$.

### Вероятностная модель
$z \in \mathbb{R}^d$ - локальная латентная переменная, т. е. своя для каждого объекта $x$.

Генеративный процесс вариационного автокодировщика:
1. Сэмплируем $z \sim p(z)$.
2. Сэмплируем $x \sim p_\theta(x | z)$.

Параметры распределения $p_\theta(x | z)$ задаются нейросетью с весами $\theta$, получающей на вход вектор $z$. Эта сеть называется генеративной сетью (generator) или декодером (decoder).

Индуцированная генеративным процессом плотность вероятности объекта $x$:

$$p_\theta(x) = \mathbb{E}_{z \sim p(z)} p_\theta(x | z)$$

### Параметризация модели
Априорное распределение на скрытые перменные - стандартное нормальное распределение: $p(z) = \mathcal{N}(z | 0, I)$.

Распределения на компоненты $x$ условно независимы относительно $z$: $p_\theta(x | z) = \prod\limits_{i = 1}^D p_\theta(x_i | z)$.

Если i-ый признак объекта вещественный, то $p_\theta(x_i | z) = \mathcal{N}(x_i | \mu_i(z, \theta), \sigma^2_i(z, \theta))$.
Здесь $\mu(z, \theta)$ и $\sigma(z, \theta)$ - детерминированные функции, задаваемые нейросетями с параметрами $\theta$.

Если i-ый признак категориальный, то $p_\theta(x_i | z) = Cat(Softmax(\omega_i(z, \theta)))$, где $\omega_i(z, \theta)$ - тоже детерминированная функция задаваемая нейросетью.

Отдельно можно рассмотреть бинарные признаки, для которых категориальное распределение превращается в распределение Бернулли с одним параметром.

### Вариационная нижняя оценка логарифма правдоподобия

Для максимизации правдоподобия максимизируем вариационную нижнюю оценку на логарифм правдоподобия:
$$\log p_\theta(x) = \mathbb{E}_{z \sim q_\phi(z | x)} \log p_\theta(x) = 
\mathbb{E}_{z \sim q_\phi(z | x)} \log \frac{p_\theta(x, z) q_\phi(z | x)}{q_\phi(z | x) p_\theta(z | x)} = 
\mathbb{E}_{z \sim q_\phi(z | x)} \log \frac{p_\theta(x, z)}{q_\phi(z | x)} + KL(q_\phi(z | x) || p_\theta(z | x))$$
$$\log p_\theta(x) \geqslant \mathbb{E}_{z \sim q_\phi(z | x)} \log \frac{p_\theta(x | z)p(z)}{q_\phi(z | x)} = 
\mathbb{E}_{z \sim q_\phi(z | x)} \log p_\theta(x | z) - KL(q_\phi(z | x) || p(z)) = L(x; \phi, \theta)
\to \max\limits_{\phi, \theta}$$

$q_\phi(z | x)$ называется предложным (proposal), распознающим (recognition) или вариационным (variational) распределением. Это гауссиана, чьи параметры задаются нейросетью с весами $\phi$:
$q_\phi(z | x) = \mathcal{N}(z | \mu_\phi(x), \sigma^2_\phi(x)I)$.
Обычно нейросеть моделирует не $\sigma_\phi(x)$, а $\log\sigma_\phi(x)$ или $\log(\exp(\sigma_\phi(x) - 1))$ или другую величину, более инвариантную к масштабу и определенную на всех вещественных числах так, чтобы $\sigma_\phi(x)$ было всегда положительным. В этом задании нужно использовать $\log(\exp(\sigma_\phi(x) - 1))$, поскольку обратное к нему преобразование softplus более стабильно, чем экспонента.

Зазор между вариационной нижней оценкой $L(x; \phi, \theta)$ на логарифм правдоподобия модели и самим логарифмом правдоподобия $\log p_\theta(x)$ - это KL-дивергенция между предолжным и апостериорным распределением на $z$: $KL(q_\phi(z | x) || p_\theta(z | x))$. Максимальное значение $L(x; \phi, \theta)$ при фиксированных параметрах модели $\theta$ достигается при $q_\phi(z | x) = p_\theta(z | x)$, но явное вычисление $p_\theta(z | x)$ требует слишком большого числа ресурсов, поэтому вместо этого вычисления вариационная нижняя оценка оптимизируется также по $\phi$. Чем ближе $q_\phi(z | x)$ к $p_\theta(z | x)$, тем точнее вариационная нижняя оценка.
Истинное апостериорное распределение $p_\theta(z | x)$ часто не может быть представлено одной гауссианой, поэтому зазор между нижней оценкой и логарифмом правдоподобия не достигает $0$. Тем не менее, есть статьи, утверждающие, что этот зазор практически не влияет на процесс оптимизации модели и его результат по сравнению с другими факторами (см. литературу).

Первое слагаемое вариационной нижней оценки $\mathbb{E}_{z \sim q_\phi(z | x)} \log p_\theta(x | z)$ называется ошибкой восстановления (reconstruction loss).
Модель, соответствующая этой части - это автокодирощик с одним стохастическим слоем, пытающийся восстановить входной объект $x$.
Если распределение $q_\phi(z | x)$ - дельта-функция, то автокодировщик со стохастическим слоем превращается в обычный автокодировщик.
Поэтому $q_\phi(z | x)$ и $p_\theta(x | z)$ иногда называют энкодером и декодером соответственно.

Слагаемое $KL(q_\phi(z | x) || p(z))$ иногда называют регуляризатором.
Оно вынуждает $z \sim q_\phi(z | x)$ быть близким к $0$ и $q_\phi(z | x)$ быть близким к $p_\theta(z | x)$.
Иногда коэффициент при KL-дивергенции полагают не равным единице или даже используют другой регуляризатор.
Естественно, после этого обучение модели перестает соответствовать максимизации правдоподобия вышеописанной вероятностной модели данных.
Это существенно снижает интерпретируемость модели, устраняет теоретические гарантии для неё.
KL-дивергенция между двумя нормальными распределениями может быть вычислена аналитически.

Для максимизации $L(x; \phi, \theta)$ используется стохастический градиентный подъем.
Градиент ошибки восстановления по $\theta$ вычисляется с помощью метода обратного распространения ошибки.
$$\frac{\partial}{\partial \theta} L(x; \phi, \theta) = \mathbb{E}_{z \sim q_\phi(z | x)} \frac{\partial}{\partial \theta} \log p_\theta(x | z)$$

Градиент KL-дивергенции по $\phi$ может быть вычислен аналитически.
Для вычисления градиента ошибки восстановления по $\phi$ используется репараметризация (reparametrization trick):
$$\varepsilon \sim \mathcal{N}(\varepsilon | 0, I)$$
$$z = \mu + \sigma \varepsilon \Rightarrow z \sim \mathcal{N}(z | \mu, \sigma^2I)$$
$$\frac{\partial}{\partial \phi} L(x; \phi, \theta) = \mathbb{E}_{\varepsilon \sim \mathcal{N}(\varepsilon | 0, I)} \frac{\partial}{\partial \phi} \log p_\theta(x | \mu_\phi(x) + \sigma_\phi(x) \varepsilon) - \frac{\partial}{\partial \phi} KL(q_\phi(z | x) || p(z))$$

Подсказка: для выведения аналиической формулы KL-дивергенции между нормальными распределениями главное - никогда не писать знак интеграла. В задании рассматриваются только нормальные распределения с диагональной матрицей ковариации, поэтому достаточно вывести KL-дивергенцию между двумя одномерными нормальными распределениями. Все, что требуется для выведения формулы:
$$KL(q || p) = \mathbb{E}_{z \sim q} \log\frac{q(z)}{p(z)}$$
$$\log \mathrm{N}(z | \mu, \sigma^2) = \frac{1}{\sqrt{2\pi}\sigma} \exp \left(-\frac{(z - \mu)^2}{2\sigma^2} \right)$$
$$\mathbb{E}_{z \sim N(\mu, \sigma)}z = \mu$$
$$\mathbb{E}_{z \sim N(\mu, \sigma)}z^2 = \mu^2 + \sigma ^ 2$$
Кстати, у трех последних формул есть часто используемые многомерные обощения.

### Оценка правдоподобия модели

Правдоподобие модели $p_\theta(x) = \mathbb{E}_{z \sim p(z)} p_\theta(x | z)$ оценивают на отложенной выборке.

Оценка может быть получена с помощью метода Монте-Карло:

$$z_i \sim p(z), i = 1, \dots, K$$
$$p_\theta(x) \approx \frac{1}{K} \sum\limits_{i = 1}^K p_\theta(x | z_i)$$

Альтернативный способ оценки - метод importance sampling. В качестве предложного распределения метода используется предложное распределение модели. Известно, что хороший выбор предложного распределения уменьшает дисперсию оценки. Для вариационных автокодировщиков оценки Монте-Карло, основанные на малом числе сэмплов, обычно занижены. Поэтому imporatance sampling также позволяет получить более высокую и точную оценку правдоподобия с помощью меньшего числа сэмплов.

$$z_i \sim q_\phi(z | x), i = 1, \dots, K$$
$$p_\theta(x) = \mathbb{E}_{z \sim p(z)} p_\theta(x | z) = \mathbb{E}_{z \sim q_\phi(z | x)} \frac{p_\theta(x | z) p(z)}{q_\phi(z | x)} \approx \frac{1}{K} \sum\limits_{i = 1}^K \frac{p_\theta(x | z_i) p(z_i)}{q_\phi(z_i | x)}$$

Для оценки логарифма правдоподобия усреднение вероятностей происходит под логарифмом:
$$\log p_\theta(x) \approx \log \frac{1}{K} \sum\limits_{i = 1}^K p_\theta(x | z_i),\,\,\,\,z_i \sim p(z)$$
$$\log p_\theta(x) \approx \log \frac{1}{K} \sum\limits_{i = 1}^K \frac{p_\theta(x | z_i) p(z_i)}{q_\phi(z_i | x)},\,\,\,\,z_i \sim q_\phi(z | x)$$
Заметим, что оценки логарифма правдоподобия уже не являются несмещенными, в отличие от оценок самого правдоподобия.
Несмотря на это, первую оценку все равно обычно называют оценкой Монте-Карло логарифма правдоподобия.
Вторая оценка известна в литературе как IWAE оценка по названию модели Importance Weighted Variational Autoencoders, в которой предлагается напрямую оптимизировать эту оценку правдоподобия для обучения автокодировщика.

### Литература
1. Auto-Encoding Variational Bayes https://arxiv.org/pdf/1312.6114.pdf, Stochastic Backpropagation and Approximate Inference in Deep Generative Models https://arxiv.org/pdf/1401.4082.pdf - оригинальные статьи про вариационный автокодировщик (две группы авторов независимо и почти одновременно предложили одинаковые модели).
2. Learning Structured Output Representation using Deep Conditional Generative Models https://papers.nips.cc/paper/5775-learning-structured-output-representation-using-deep-conditional-generative-models.pdf - обусловленный вариационный автокодировщик для генерации из обусловленного распределения на объекты.
3. Importance Weighted Autoencoders https://arxiv.org/pdf/1509.00519.pdf - вариационный автокодировщик, оптимизирующий более точную нижнюю оценку на логарифм правдоподобия.
4. Tighter Variational Bounds are Not Necessarily Better https://arxiv.org/pdf/1802.04537.pdf - статья, показывающая, что предыдущая статья ухудшает обучение предложной сети автокодировщика как следствие более хорошей нижней оценки, и предлагающая способы решения этой проблемы.
5. Variational Inference with Normalizing Flows https://arxiv.org/pdf/1505.05770.pdf, Improved Variational Inference with Inverse Autoregressive Flow http://papers.nips.cc/paper/6581-improved-variational-inference-with-inverse-autoregressive-flow.pdf - более богатые семейства предложных распредлений.
6. VAE with a VampPrior https://arxiv.org/pdf/1705.07120.pdf - обучение априорного распределения на скрытые представления совместно с предложным. Улучшает правдоподобие модели, но делает менее интерпретируемым скрытое пространство.
7. Ladder Variation Autoencoders http://papers.nips.cc/paper/6275-ladder-variational-autoencoders.pdf - теперь у каждого объекта есть не одно скрытое представление, а несколько организованных в иерархию.
8. Inference Suboptimality in Variational Autoencoders https://arxiv.org/pdf/1801.03558.pdf - утверждает, что отличие предложного распределения от истинного апостериорного распределения в скрытом пространстве вызваны недостаточной выразительностью предложной сети, а не бедным семейством предложных распределений. Богатое же семейство предложных моделей снижает требования к выразительности предложной сети.

## Практика

### Загрузка, предобработка и визуалиация данных

In [ ]:
from torchvision.datasets import MNIST
from torch.utils.data import TensorDataset, DataLoader
import torch
from torch import nn
from torch import optim

In [ ]:
data = MNIST('mnist', download=True, train=True)
train_data = TensorDataset(data.train_data.view(-1, 28 * 28).float() / 255, data.train_labels)
data = MNIST('mnist', download=True, train=False)
test_data = TensorDataset(data.test_data.view(-1, 28 * 28).float() / 255, data.test_labels)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

def show_images(x):
    plt.figure(figsize=(12, 12 / 10 * (x.shape[0] // 10 + 1)))
    x = x.view(-1, 28, 28)
    for i in range(x.shape[0]):
        plt.subplot(x.shape[0] // 10 + 1, 10, i + 1)
        plt.imshow(x.data[i].numpy(), cmap='Greys_r', vmin=0, vmax=1, interpolation='lanczos')
        plt.axis('off')

show_images(train_data[:10][0])

Для корректности вероятностной модели исходное изображение также должно быть бинаризовано.

Бинаризация может производиться как округлением данных в датасете, так и сэмплированием из распределения Бернулли каждого пикселя. Округление приводит к более гладким фигурам в обучающей выборке, поэтому будем использовать его.

In [ ]:
show_images(torch.bernoulli(train_data[:10][0]))

In [ ]:
show_images(train_data[:10][0].round())

In [ ]:
train_data.tensors = (train_data.tensors[0].round(), train_data.tensors[1])
test_data.tensors = (test_data.tensors[0].round(), test_data.tensors[1])

### Вспомогательные функции для обучения и тестирования

In [ ]:
def model_test_loss(compute_loss, batch_size=100, max_batches=None, verbose=False):
    """
    Функция вычисляет усредненное значение функции потерь по тестовым данным.
    Вход: compute_loss, функция, принимающая батч в виде матрицы torch.FloatTensor
    и возвращающая float - функцию потерь на батче.
    Вход: batch_size, int.
    Вход: max_batches, int - если задано, включает режим оценки функции потерь
    с помощью сэмплирования батчей вместо полного прохода по данным и указывает,
    после какого батча прекратить вычисления.
    Вход: verbose, bool - указывает, печатать ли текущее состояние в процессе работы.
    Возвращаемое значение: float - оценка функции потерь на тестовых данных.
    """
    dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=(max_batches is None))
    num_batches = len(dataloader)
    avg_loss = 0
    for i, (batch, _) in enumerate(dataloader):
        loss = compute_loss(batch)
        avg_loss += (loss - avg_loss) / (i + 1)
        if verbose and (i + 1) % 10 == 0:
            print('\rTest loss:', avg_loss,
                  'Batch', i + 1, 'of', num_batches, ' ' * 10, end='', flush=True)
        if verbose and (i + 1) % 100 == 0:
            print(flush=True)
        if max_batches and i >= max_batches:
            break
    return avg_loss

In [ ]:
def train_model(model, tests=[], batch_size=100, num_epochs=5, learning_rate=1e-3, maximization=True):
    """
    Обучает модель.
    Вход: model, Module - объект, модель.
    У этого объекта должна быть функция batch_loss от batch - FloatTensor и K - int,
    возвращающая скаляр Variable - функцию потерь на батче, которая должна быть
    оптимизирована.
    Вход: tests - список тестов, выполняемых после каждого 100-го батча.
    Каждый элемент списка - словарь с полями 'name' - уникальным идентификатором
    теста и 'func' - функцией от модели.
    Вход: batch_size, int.
    Вход: num_epochs, int.
    Вход: learning_rate, float.
    Возвращаемое значение: словарь с полями 'model' - обученная модель,
    'train_losses_list' - список функций потерь на каждом батче и 
    'test_results' - список результатов тестирования. Каждый результат
    тестирования - словарь вида name: value, где name - имя теста,
    value - результат его выполнения.
    """
    gd = optim.Adam(model.parameters(), lr=learning_rate)
    dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    train_losses = []
    test_results = []
    for _ in range(num_epochs):
        for i, (batch, _) in enumerate(dataloader):
            total = len(dataloader)
            loss = model.batch_loss(batch)
            if maximization:
                (-loss).backward()
            else:
                loss.backward()
            train_losses.append(float(loss))
            if (i + 1) % 10 == 0:
                print('\rTrain loss:', train_losses[-1],
                      'Batch', i + 1, 'of', total, ' ' * 10, end='', flush=True)
            if (i + 1) % 100 == 0:
                cur_test_result = {}
                for test in tests:
                    cur_test_result[test['name']] = test['func'](model)
                test_results.append(cur_test_result)
                print(flush=True)
            gd.step()
            gd.zero_grad()
    return {
        'model': model,
        'train_losses_list': train_losses,
        'test_results': test_results
    }

In [ ]:
n = 15
digit_size = 28

from scipy.stats import norm
import numpy as np

grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

def draw_manifold(generator):
    figure = np.zeros((digit_size * n, digit_size * n))
    for i, yi in enumerate(grid_x):
        for j, xi in enumerate(grid_y):
            z_sample = np.array([[xi, yi]])

            x_decoded = generator(z_sample)
            digit = x_decoded
            figure[i * digit_size: (i + 1) * digit_size,
                   j * digit_size: (j + 1) * digit_size] = digit
    plt.figure(figsize=(10, 10))
    plt.imshow(figure, cmap='Greys_r', vmin=0, vmax=1, interpolation='lanczos')
    plt.axis('off')
    plt.show()

In [ ]:
def draw_latent_space(data, target, encoder):
    z_test = encoder(data)
    plt.figure(figsize=(7, 6))
    plt.scatter(z_test[:, 0], z_test[:, 1], c=target, cmap='gist_rainbow', alpha=0.75)
    plt.colorbar()
    plt.show()

In [ ]:
from sklearn.manifold import TSNE

## Обычный автокодировщик

In [ ]:
from models import AE

### Обучение моделей

In [ ]:
ae_tests = [
    {
        'name': 'test_loss',
        'func': lambda model:
                model_test_loss(lambda batch:
                                float(model.batch_loss(batch)),
                                max_batches=20)
    }
]

In [ ]:
ae_model_d2 = train_model(AE(2, 784), tests=ae_tests, maximization=False, num_epochs=50)

In [ ]:
ae_model_d10 = train_model(AE(10, 784), tests=ae_tests, maximization=False, num_epochs=50)

### Оценка качества моделей
Визуальная оценка генерируемых объектов

In [ ]:
show_images(ae_model_d2['model'].generate_samples(20))

In [ ]:
show_images(ae_model_d10['model'].generate_samples(20))

Визуализация латентного пространства (с точки зрения декодера)

In [ ]:
def draw_manifold_ae(model):
    generator = lambda z: model.decode(torch.from_numpy(z).float()).view(28, 28).data.numpy()
    return draw_manifold(generator)

In [ ]:
draw_manifold_ae(ae_model_d2['model'])

Визуализация латентного пространства (с точки зрения энкодера)

In [ ]:
draw_latent_space(test_data.tensors[0][::10], test_data.tensors[1][::10],
                  lambda data: ae_model_d2['model'].encode(data).detach())

In [ ]:
ae_encoder_d10 = lambda data: TSNE().fit_transform(ae_model_d10['model'].encode(data).data.numpy())
draw_latent_space(test_data.tensors[0][::25], test_data.tensors[1][::25], ae_encoder_d10)

## Автокодировщик: теперь вариационный!

В качестве функции потерь используем бинарную кроссэнтропию.
Это означает, что мы предполагаем, что каждый пискель - бинарная случайная величина.
Генеративная сеть выдает вероятность каждого пикселя быть равным $1$.

In [ ]:
from models import log_likelihood, log_mean_exp, kl

In [ ]:
from models import VAE

### Оценки функции потерь

In [ ]:
from models import gaussian_log_pdf, compute_log_likelihood_monte_carlo, compute_log_likelihood_iwae

### Обучение модели

In [ ]:
vae_tests = [
    {
        'name': 'MC',
        'func': lambda model:
                model_test_loss(lambda batch:
                                compute_log_likelihood_monte_carlo(batch, model, K=10),
                                max_batches=20)
    },
    {
        'name': 'IS',
        'func': lambda model:
                model_test_loss(lambda batch:
                                compute_log_likelihood_iwae(batch, model, K=10),
                                max_batches=20)
    }
]

In [ ]:
vae_model_d2 = train_model(VAE(2, 784), tests=vae_tests, num_epochs=50)

In [ ]:
vae_model_d10 = train_model(VAE(10, 784), tests=vae_tests, num_epochs=50)

In [ ]:
torch.save(vae_model_d10['model'].state_dict(), 'vae_d10.model')

### Оценка качества модели

Визуальная оценка генерируемых объектов

In [ ]:
show_images(vae_model_d2['model'].generate_samples(20))

In [ ]:
show_images(vae_model_d10['model'].generate_samples(20))

Визуализация латентного пространства (с точки зрения декодера)

In [ ]:
def draw_manifold_vae(model):
    generator = lambda z: model.generative_distr(torch.from_numpy(z).float()).view(28, 28).data.numpy()
    return draw_manifold(generator)

In [ ]:
draw_manifold_vae(vae_model_d2['model'])

Визуализация латетного пространства (с точки зрения энкодера)

In [ ]:
vae_encoder = lambda data, model: model.sample_latent(model.proposal_distr(data))[:, 0].detach()
draw_latent_space(test_data.tensors[0][::10], test_data.tensors[1][::10],
                  lambda data: vae_encoder(data, vae_model_d2['model']))

In [ ]:
vae_encoder_d10 = lambda data: TSNE().fit_transform(vae_encoder(data, vae_model_d10['model']).data.numpy())
draw_latent_space(test_data.tensors[0][::25], test_data.tensors[1][::25], vae_encoder_d10)

### Оценки логарифма правдоподобия на тестовых данных

In [ ]:
plt.figure(figsize=(9, 6))
for label, name, model in [
    ('VAE, Monte-Carlo, $d = 2$', 'MC', vae_model_d2),
    ('VAE, Monte-Carlo, $d = 10$', 'MC', vae_model_d10),
    ('VAE, Importance Sampling, $d = 2$', 'IS', vae_model_d2),
    ('VAE, Importance Sampling, $d = 10$', 'IS', vae_model_d10),
]:
    data = [x[name] for x in model['test_results']]
    x_labels = (1 + np.arange(len(data))) / 6
    plt.plot(x_labels, data, label=label)
plt.xlabel('Epoch')
plt.xlim(xmax=x_labels[-1])
plt.ylabel('Log-likelihood estimation, 10 samples')
plt.legend()
pass

In [ ]:
test_results = []
for K in [1, 5, 10, 50, 100, 500, 1000]:
    print('K =', K, flush=True)
    vae_tests_sampling = [
        {
            'name': 'D10MC',
            'func': model_test_loss(lambda batch:
                                    compute_log_likelihood_monte_carlo(batch, vae_model_d10['model'], K=K),
                                    batch_size=10,
                                    max_batches=50)
        },
        {
            'name': 'D10IS',
            'func': model_test_loss(lambda batch:
                                    compute_log_likelihood_iwae(batch, vae_model_d10['model'], K=K),
                                    batch_size=10,
                                    max_batches=50)
        },
        {
            'name': 'D2MC',
            'func': model_test_loss(lambda batch:
                                    compute_log_likelihood_monte_carlo(batch, vae_model_d2['model'], K=K),
                                    batch_size=10,
                                    max_batches=50)
        },
        {
            'name': 'D2IS',
            'func': model_test_loss(lambda batch:
                                    compute_log_likelihood_iwae(batch, vae_model_d2['model'], K=K),
                                    batch_size=10,
                                    max_batches=50)
        }
    ]
    cur_test_results = {'K': K}
    for test in vae_tests_sampling:
        cur_test_results[test['name']] = test['func']
    test_results.append(cur_test_results)

In [ ]:
plt.figure(figsize=(9, 6))
for label, name in [
    ('VAE, Monte-Carlo, $d = 2$', 'D2MC'),
    ('VAE, Importance Sampling, $d = 2$', 'D2IWAE'),
    ('VAE, Monte-Carlo, $d = 10$', 'D10MC'),
    ('VAE, Importance Sampling, $d = 10$', 'D10IWAE'),
]:
    data = [x[name] for x in test_results]
    x_labels = [x['K'] for x in test_results]
    plt.plot(x_labels, data, label=label)
plt.xlabel('Number of samples')
plt.xscale('log')
plt.ylabel('Log-likelihood estimation')
plt.legend()
pass

## Выводы

Место для ваших выводов, наблюдений, гипотез.